# Interactive Lennard-Jones Potential Exploration

This notebook provides an interactive introduction to the Lennard-Jones potential,
one of the fundamental interactions in molecular dynamics simulations.

## Learning Objectives

1. Understand the Lennard-Jones (LJ) potential energy function
2. Explore how LJ parameters (σ, ε) affect the potential
3. Visualize forces derived from the LJ potential
4. Run simple 2D simulations with LJ interactions
5. Connect theory to observable behavior

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider

# Add parent directory to path
sys.path.insert(0, os.path.join(os.getcwd(), '..'))

from core import Simulation2D, Renderer2D, LennardJones2D

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)

## Part 1: The Lennard-Jones Potential

The Lennard-Jones potential describes the interaction between two uncharged particles:

$$E(r) = 4\epsilon \left[\left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6\right]$$

Where:
- **r**: Distance between particles
- **ε** (epsilon): Depth of the potential well (energy units)
- **σ** (sigma): Distance where potential crosses zero

### Physical Meaning

- **r⁻¹² term**: Short-range **repulsion** (Pauli exclusion)
- **r⁻⁶ term**: Long-range **attraction** (van der Waals/dispersion forces)

GROMOS uses an alternative form:

$$E(r) = \frac{C_{12}}{r^{12}} - \frac{C_6}{r^6}$$

Where $C_{12} = 4\epsilon\sigma^{12}$ and $C_6 = 4\epsilon\sigma^6$

In [ ]:
# Create LJ potential object
lj = LennardJones2D(epsilon=1.0, sigma=0.3)

# Plot energy and force
r_values = np.linspace(0.25, 1.0, 200)
energies = [lj.energy(r) for r in r_values]
forces = [lj.force_magnitude(r) for r in r_values]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Energy plot
ax1.plot(r_values, energies, 'b-', linewidth=2, label='LJ Potential')
ax1.axhline(0, color='k', linestyle='--', alpha=0.3)
ax1.axvline(lj.sigma, color='g', linestyle=':', alpha=0.5, label=f'σ = {lj.sigma} nm')

r_min, e_min = lj.minimum_energy()
ax1.plot(r_min, e_min, 'ro', markersize=10, label=f'Minimum: r={r_min:.3f} nm, E={e_min:.3f} kJ/mol')

ax1.set_xlabel('Distance r (nm)', fontsize=12)
ax1.set_ylabel('Energy (kJ/mol)', fontsize=12)
ax1.set_title('Lennard-Jones Potential Energy', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(-2, 3)

# Force plot
ax2.plot(r_values, forces, 'r-', linewidth=2, label='LJ Force')
ax2.axhline(0, color='k', linestyle='--', alpha=0.3)
ax2.axvline(r_min, color='orange', linestyle=':', alpha=0.5, label=f'Zero force at r={r_min:.3f} nm')

ax2.set_xlabel('Distance r (nm)', fontsize=12)
ax2.set_ylabel('Force (kJ/mol/nm)', fontsize=12)
ax2.set_title('Lennard-Jones Force', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nKey Properties:")
print(f"  ε (epsilon): {lj.epsilon} kJ/mol")
print(f"  σ (sigma): {lj.sigma} nm")
print(f"  Minimum energy: {e_min:.3f} kJ/mol at r = {r_min:.3f} nm")
print(f"  Zero crossing: r = {lj.sigma:.3f} nm")
print(f"\nInterpretation:")
print(f"  r < {r_min:.3f} nm: Repulsive (particles push apart)")
print(f"  r = {r_min:.3f} nm: Equilibrium (zero net force)")
print(f"  r > {r_min:.3f} nm: Attractive (particles pull together)")

## Part 2: Exploring Parameter Effects

Let's see how changing ε and σ affects the potential:

In [ ]:
# Compare different parameters
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

r_values = np.linspace(0.2, 1.2, 200)

# Varying epsilon (well depth)
for eps in [0.5, 1.0, 2.0]:
    lj_temp = LennardJones2D(epsilon=eps, sigma=0.3)
    energies = [lj_temp.energy(r) for r in r_values]
    ax1.plot(r_values, energies, linewidth=2, label=f'ε = {eps} kJ/mol')

ax1.axhline(0, color='k', linestyle='--', alpha=0.3)
ax1.set_xlabel('Distance r (nm)', fontsize=12)
ax1.set_ylabel('Energy (kJ/mol)', fontsize=12)
ax1.set_title('Effect of ε (Well Depth)', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(-3, 3)

# Varying sigma (particle size)
for sig in [0.25, 0.3, 0.35]:
    lj_temp = LennardJones2D(epsilon=1.0, sigma=sig)
    energies = [lj_temp.energy(r) for r in r_values]
    ax2.plot(r_values, energies, linewidth=2, label=f'σ = {sig} nm')

ax2.axhline(0, color='k', linestyle='--', alpha=0.3)
ax2.set_xlabel('Distance r (nm)', fontsize=12)
ax2.set_ylabel('Energy (kJ/mol)', fontsize=12)
ax2.set_title('Effect of σ (Particle Size)', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(-2, 2)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("  - Larger ε → Deeper well → Stronger attraction")
print("  - Larger σ → Minimum at larger distance → Bigger particles")

## Part 3: Two-Particle Collision Simulation

Let's simulate two particles approaching each other and see the LJ potential in action:

In [ ]:
# Create a simple 2-particle simulation
sim = Simulation2D(
    box_size=(3.0, 2.0),
    periodic=False,
    integrator='velocity-verlet',
    dt=0.001
)

# Add two particles approaching each other
sim.add_particle(position=[0.5, 1.0], velocity=[2.0, 0.0], color='blue')
sim.add_particle(position=[2.5, 1.0], velocity=[-2.0, 0.0], color='red')

# Track distance and energy
distances = []
pot_energies = []
kin_energies = []

print("Running collision simulation...")

for step in range(800):
    sim.integrate_step()
    
    # Calculate distance between particles
    pos1 = sim.particles[0].position
    pos2 = sim.particles[1].position
    dist = np.linalg.norm(pos2 - pos1)
    distances.append(dist)
    
    # Track energies
    ke, pe, _ = sim.total_energy()
    pot_energies.append(pe)
    kin_energies.append(ke)

# Plot results
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

times = np.arange(len(distances)) * sim.dt

# Distance vs time
ax1.plot(times, distances, 'b-', linewidth=2)
ax1.axhline(lj.sigma, color='g', linestyle='--', label=f'σ = {lj.sigma} nm')
r_min, _ = lj.minimum_energy()
ax1.axhline(r_min, color='orange', linestyle='--', label=f'r_min = {r_min:.3f} nm')
ax1.set_xlabel('Time (ps)', fontsize=12)
ax1.set_ylabel('Distance (nm)', fontsize=12)
ax1.set_title('Particle Separation During Collision', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Energy vs time
ax2.plot(times, kin_energies, 'b-', linewidth=2, label='Kinetic Energy')
ax2.plot(times, pot_energies, 'r-', linewidth=2, label='Potential Energy')
total_energies = np.array(kin_energies) + np.array(pot_energies)
ax2.plot(times, total_energies, 'k--', linewidth=2, label='Total Energy')
ax2.set_xlabel('Time (ps)', fontsize=12)
ax2.set_ylabel('Energy (kJ/mol)', fontsize=12)
ax2.set_title('Energy Exchange During Collision', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nWhat happened?")
print("1. Particles approach with kinetic energy")
print("2. LJ attraction pulls them together (KE increases)")
print("3. At close range, repulsion dominates")
print("4. Particles bounce off each other")
print("5. Energy converts between kinetic and potential")
print("6. Total energy is conserved!")

## Part 4: Many-Particle System

Now let's create a small gas of LJ particles and watch them interact:

In [ ]:
# Create a small system
N = 16
box_size = (3.0, 3.0)

sim = Simulation2D(
    box_size=box_size,
    periodic=True,
    integrator='velocity-verlet',
    dt=0.002
)

# Initialize in grid
grid_size = int(np.ceil(np.sqrt(N)))
spacing = min(box_size) / grid_size

for i in range(N):
    ix = i % grid_size
    iy = i // grid_size
    x = (ix + 0.5) * spacing
    y = (iy + 0.5) * spacing
    
    color = ['blue', 'red', 'green', 'orange'][i % 4]
    sim.add_particle(position=[x, y], color=color)

# Initialize velocities at 300 K
sim.initialize_velocities(300.0)
sim.set_temperature(300.0, tau=0.1)

# Run simulation and collect data
print("Running many-particle simulation...")

for step in range(500):
    sim.integrate_step()

# Plot energy evolution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

times = np.arange(len(sim.energies['total'])) * sim.dt

axes[0].plot(times, sim.energies['kinetic'], 'b-', linewidth=1.5, label='Kinetic')
axes[0].plot(times, sim.energies['potential'], 'r-', linewidth=1.5, label='Potential')
axes[0].plot(times, sim.energies['total'], 'k-', linewidth=2, label='Total')
axes[0].set_xlabel('Time (ps)')
axes[0].set_ylabel('Energy (kJ/mol)')
axes[0].set_title('Energy Evolution')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Temperature
k_B = 0.00831446261815324
n_df = 2 * N
temps = [2 * ke / (k_B * n_df) if n_df > 0 else 0 for ke in sim.energies['kinetic']]

axes[1].plot(times, temps, 'orange', linewidth=2)
axes[1].axhline(300, color='r', linestyle='--', label='Target: 300 K')
axes[1].set_xlabel('Time (ps)')
axes[1].set_ylabel('Temperature (K)')
axes[1].set_title('Temperature Evolution')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal statistics:")
print(f"  Temperature: {sim.temperature():.1f} K")
print(f"  LJ energy per particle: {sim.energies['lj'][-1]/N:.3f} kJ/mol")
print(f"  Total energy: {sim.energies['total'][-1]:.2f} kJ/mol")

## Summary and Key Takeaways

1. **Lennard-Jones potential** is a simple but effective model for uncharged particle interactions
2. **Two competing terms**: r⁻¹² (repulsion) and r⁻⁶ (attraction)
3. **Parameters control behavior**:
   - ε (epsilon): Interaction strength
   - σ (sigma): Effective particle size
4. **Energy conservation**: KE ↔ PE during collisions
5. **Temperature** emerges from kinetic energy of many particles

## Next Steps

Try the other example scripts to see:
- Gas in a box (ideal gas behavior)
- Liquid droplet formation (phase separation)
- Crystal lattice (ordered structures)
- Polymer chains (bonded systems)
- Protein folding (complex interactions)
